# Kayak Project ⛵
Summary :
- Collect cities informations
- Collect weather informations
- Chose 5 best cities with the best weather
- Create the spider 
- Storing data in a RDS DataWarehouse and test it

In [1]:
# Import usual libraries
import pandas as pd
import requests
import json
import numpy as np
import plotly.express as px

import boto3
from io import StringIO

import os
from dotenv import load_dotenv
load_dotenv(dotenv_path=r'C:\Users\ambri\Desktop\vscodetraining\Training\FullStack\.env')

from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

## Collect cities informations

In [2]:
# Cities to study
cities = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille","Aix en Provence","Avignon","Uzes","Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]

In [3]:
# Collect data on longitudes and latitudes
url = 'https://nominatim.openstreetmap.org/search'
cities_info = []

for city in cities:

    payload = {
        'city' : city,
        'country' : 'france',
        'country-code' : 'fr',
        'format' : 'json'
        }
    headers = {
        'User-Agent': "Chrome/97.0"
    }
    
    r_city = requests.get(url, 
                          params=payload,headers=headers)
    
    if r_city.status_code == 200 :
        json_data = r_city.json

        if json_data :
            cities_info.append({'city': city,
                        'lat' : float(r_city.json()[0]['lat']),
                        'lon' : float(r_city.json()[0]['lon']),
    })
        else :
            print("No data found")
    else :
        print(f"Error :{r_city.status_code}")

In [4]:
#Dataframe with all the cities, long and lat
df_cities = pd.DataFrame.from_dict(cities_info)
df_cities

,city,lat,lon
0,Mont Saint Michel,48.635954,-1.511460
1,St Malo,48.649518,-2.026041
2,Bayeux,49.276462,-0.702474
3,Le Havre,49.493898,0.107973
4,Rouen,49.440459,1.093966
5,Paris,48.853495,2.348391
6,Amiens,49.894171,2.295695
7,Lille,50.636565,3.063528
8,Strasbourg,48.584614,7.750713
9,Chateau du Haut Koenigsbourg,48.249523,7.345492


In [5]:
# Map with all the cities
fig = px.scatter_map(df_cities,
                        lat='lat',
                        lon='lon',
                        center=dict(lat=46.2, lon=2.2),
                        zoom=4.5,
                        hover_name='city',
                        title='Map of the 35 cities',
                        height=700,
                        width=700
                        )
fig.update_layout(margin={"r":10,"t":100,"l":10,"b":10})
fig.show()
fig.write_html("cities.html")

In [6]:
# Load the dataframe in a data lake : S3 bucket
session = boto3.Session(aws_access_key_id=os.environ["AWS_ACCESS_KEY"], 
                        aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])

s3 = session.resource("s3")

bucket = s3.create_bucket(Bucket="citiesinfoskayakjedha")

# Transform the dataframe into a csv. and upload the CSV file to S3
csv = df_cities.to_csv()
put_object = bucket.put_object(Key="cities.csv", Body=csv)


## Collect weather informations

In [7]:
# Collect data about the forecast weather
import os

APIkey = os.environ['API_KEY_WEATHER']
weather_info = []

for i in range(len(df_cities)) :

    lat = df_cities.loc[i, 'lat']
    lon = df_cities.loc[i, 'lon']

    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&exclude=daily&appid={APIkey}&units=metric"

    r_meteo = requests.get(url)
    response_meteo = r_meteo.json()
     
    weather_info.append({'lat' : lat,
                         'lon' : lon,'weather': response_meteo['weather'][0]['description'],  
                         'humidity': response_meteo['main']['humidity'],  
                         'wind_speed': response_meteo['wind']['speed'],  
                         'clouds': response_meteo['clouds']['all'],
                         'feels_like' : response_meteo['main']['feels_like'],
'city': response_meteo['name']
    })

In [8]:
# Dataframe with the weather
weather_info = pd.DataFrame(weather_info)
weather_info['city'] = cities
weather_info.head()

,lat,lon,weather,humidity,wind_speed,clouds,feels_like,city
0,48.635954,-1.511460,clear sky,82,3.85,7,-0.99,Mont Saint Michel
1,48.649518,-2.026041,clear sky,93,3.09,2,-0.82,St Malo
2,49.276462,-0.702474,few clouds,85,3.42,23,-2.47,Bayeux
3,49.493898,0.107973,mist,87,2.57,6,-0.11,Le Havre
4,49.440459,1.093966,mist,83,4.63,20,-0.57,Rouen


In [9]:
# Collect data of the forecast weather for the next days
forecast_weather = []

for city in cities:


    r = requests.get(f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={APIkey}&units=metric")

    lat, lon = [(weather_info['lat'], weather_info['lon']) for weather_info in cities_info if weather_info['city'] == city][0]

    if r.status_code == 200:
        r_json_criteria = r.json()['list']
        for timepoint in r_json_criteria:
            weather = {
                "city": city,
                'latitude': lat, 
                'longitude': lon, 
                "timepoint": timepoint["dt"],
                "main_weather": timepoint["weather"][0]["description"],
                "humidity": timepoint["main"]["humidity"],
                "feels_like": timepoint["main"]["feels_like"],
                "wind_speed": timepoint["wind"]["speed"],
            }
            forecast_weather.append(weather)
    else:
        print(f"{city} not found, skipped")

In [10]:
# Dataframe of the forecast weather
forecast_weather_df = pd.DataFrame(data = forecast_weather)

# Create unique identifier for each city
city_ids = forecast_weather_df.groupby('city').ngroup() + 1
forecast_weather_df['city_id'] = city_ids
forecast_weather_df

# Transform the dataframe into a csv. and upload the CSV file to S3
csv = forecast_weather_df.to_csv()
put_object = bucket.put_object(Key="forecast_weather_df.csv", Body=csv)

In [11]:
# Create average forecast weather dataframe
avg_numerical = forecast_weather_df.groupby('city').mean(numeric_only=True)
avg_categorical = forecast_weather_df.groupby('city')['main_weather'].agg(lambda x: x.value_counts().index[0])

avg_forecast_weather_df = pd.concat([avg_numerical,avg_categorical],axis=1).reset_index()

avg_forecast_weather_df = avg_forecast_weather_df.drop(columns=["timepoint"])
avg_forecast_weather_df

,city,latitude,longitude,humidity,feels_like,wind_speed,city_id,main_weather
0,Aigues Mortes,43.566152,4.191540,64.850,8.62200,2.94825,1.0,overcast clouds
1,Aix en Provence,43.529842,5.447474,58.175,11.80650,5.40325,2.0,overcast clouds
2,Amiens,49.894171,2.295695,58.450,7.97625,2.29875,3.0,clear sky
3,Annecy,45.899235,6.128885,76.325,5.17900,2.14325,4.0,clear sky
4,Ariege,42.845802,1.606759,81.250,8.58625,7.07800,5.0,overcast clouds
5,Avignon,43.949249,4.805901,52.675,9.44200,3.49075,6.0,overcast clouds
6,Bayeux,49.276462,-0.702474,78.675,5.70625,4.65575,7.0,clear sky
7,Bayonne,43.494514,-1.473666,76.825,10.75925,2.47125,8.0,overcast clouds
8,Besancon,47.238022,6.024362,68.350,5.50525,1.61700,9.0,clear sky
9,Biarritz,43.483252,-1.559278,69.350,9.25850,6.15850,10.0,overcast clouds


In [12]:
# Load the dataframe in a data lake : S3 bucket
session = boto3.Session(aws_access_key_id=os.environ["AWS_ACCESS_KEY"], 
                        aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])

s3 = session.resource("s3")

bucket_name = "citiesinfoskayakjedha"
bucket = s3.Bucket(bucket_name)

# Transform the dataframe into a csv. and upload the CSV file to S3
csv = avg_forecast_weather_df.to_csv()
put_object = bucket.put_object(Key="avg_forecast_weather.csv", Body=csv)

## Chose 5 best cities with the best weather

In [13]:
# We create a mask for the best conditions for photography (sky clear or a bit of cloud) and good feels like)
conditions = (avg_forecast_weather_df['humidity']<100) & (avg_forecast_weather_df['feels_like']>5) & (avg_forecast_weather_df['wind_speed']<10) & ((avg_forecast_weather_df['main_weather'] == 'clear sky') |(avg_forecast_weather_df['main_weather'] == 'overcast clouds') )
top5_cities_df = avg_forecast_weather_df[conditions].sort_values("feels_like",ascending=False).head()                                                                                                                                                                                                                                                                                                                               

In [14]:
city_top = [x for x in top5_cities_df['city']]
print(city_top)

['Aix en Provence', 'Carcassonne', 'Bayonne', 'Marseille', 'La Rochelle']


In [15]:
# Plot the 5 best cities with the feels like
fig = px.scatter_mapbox(
    top5_cities_df, 
    title="Top 5 cities for a photography week", 
    lat="latitude", 
    lon="longitude", 
    color="feels_like",
    size="humidity",
    zoom=5, 
    mapbox_style="open-street-map",
    hover_name=top5_cities_df['city'],
    hover_data = ['wind_speed', 'main_weather','feels_like','humidity'],
    opacity=1,
    height=700,
    width=700
    
)
fig.update_layout(margin={"r":10,"t":100,"l":10,"b":10})
fig.show()
fig.write_html("top_5_cities.html")

In [16]:
# Load the dataframe in a data lake : S3 bucket
session = boto3.Session(aws_access_key_id=os.environ["AWS_ACCESS_KEY"], 
                        aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])

s3 = session.resource("s3")

bucket_name = "citiesinfoskayakjedha"
bucket = s3.Bucket(bucket_name)

# Transform the dataframe into a csv. and upload the CSV file to S3
csv = top5_cities_df.to_csv()
put_object = bucket.put_object(Key="top5_cities.csv", Body=csv)

## Create the spider 

In [17]:
!python spider_booking.py

2025-03-02 09:06:55 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: scrapybot)
2025-03-02 09:06:55 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.10.0, w3lib 2.3.1, Twisted 24.11.0, Python 3.13.0 (tags/v3.13.0:60403a5, Oct  7 2024, 09:38:07) [MSC v.1941 64 bit (AMD64)], pyOpenSSL 25.0.0 (OpenSSL 3.4.0 22 Oct 2024), cryptography 44.0.0, Platform Windows-11-10.0.26100-SP0
2025-03-02 09:06:55 [scrapy.addons] INFO: Enabled addons:
[]
2025-03-02 09:06:55 [scrapy.extensions.telnet] INFO: Telnet Password: fca45c92f67bae62
2025-03-02 09:06:55 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2025-03-02 09:06:55 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20, 'USER_AGENT': 'Chrome/97.0'}
2025-03-02 09:06:56 [scrapy.middleware] INFO: Enabled downloader middlewares:
['sc

## Storing data in a RDS DataWarehouse adn test it

In [18]:
# Initialization of the SQL engine
USERNAME = "postgres"
HOSTNAME = os.environ["HOSTNAME"]
PASSWORD = os.environ["PASSWORD"]

engine = create_engine(f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}", echo=True)
Session = sessionmaker(bind=engine)
session = Session()

In [19]:
# Storing the two dataframes as SQL tables
df_cities.to_sql('cities', con=engine, if_exists='replace', index=False)
forecast_weather_df.to_sql('weather', con=engine, if_exists='replace', index=False)
avg_forecast_weather_df.to_sql('avg_forecast', con=engine, if_exists='replace', index=False)
top5_cities_df.to_sql('top_5_cities', con=engine, if_exists='replace', index=False)

2025-03-02 09:07:18,744 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-03-02 09:07:18,746 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-02 09:07:18,780 INFO sqlalchemy.engine.Engine select current_schema()
2025-03-02 09:07:18,782 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-02 09:07:18,815 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-03-02 09:07:18,816 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-02 09:07:18,849 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-02 09:07:18,858 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

5

### Test

In [20]:
# Cities
stmt = text("SELECT * FROM cities ")
conn = engine.connect()
result = conn.execute(stmt)
result.fetchall()

2025-03-02 09:07:20,572 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-02 09:07:20,573 INFO sqlalchemy.engine.Engine SELECT * FROM cities 
2025-03-02 09:07:20,574 INFO sqlalchemy.engine.Engine [generated in 0.00237s] {}


[('Mont Saint Michel', 48.6359541, -1.511459954959514),
 ('St Malo', 48.649518, -2.0260409),
 ('Bayeux', 49.2764624, -0.7024738),
 ('Le Havre', 49.4938975, 0.1079732),
 ('Rouen', 49.4404591, 1.0939658),
 ('Paris', 48.8534951, 2.3483915),
 ('Amiens', 49.8941708, 2.2956951),
 ('Lille', 50.6365654, 3.0635282),
 ('Strasbourg', 48.584614, 7.7507127),
 ('Chateau du Haut Koenigsbourg', 48.2495226, 7.3454923),
 ('Colmar', 48.0777517, 7.3579641),
 ('Eguisheim', 48.0447968, 7.3079618),
 ('Besancon', 47.2380222, 6.0243622),
 ('Dijon', 47.3215806, 5.0414701),
 ('Annecy', 45.8992348, 6.1288847),
 ('Grenoble', 45.1875602, 5.7357819),
 ('Lyon', 45.7578137, 4.8320114),
 ('Gorges du Verdon', 43.7496562, 6.3285616),
 ('Bormes les Mimosas', 43.1506968, 6.3419285),
 ('Cassis', 43.2140359, 5.5396318),
 ('Marseille', 43.2961743, 5.3699525),
 ('Aix en Provence', 43.5298424, 5.4474738),
 ('Avignon', 43.9492493, 4.8059012),
 ('Uzes', 44.0121279, 4.4196718),
 ('Nimes', 43.8374249, 4.3600687),
 ('Aigues Mortes',

In [21]:
# Weather
stmt = text("SELECT AVG(feels_like) FROM weather")
conn = engine.connect()
result = conn.execute(stmt)
result.fetchall()

2025-03-02 09:07:20,743 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-02 09:07:20,744 INFO sqlalchemy.engine.Engine SELECT AVG(feels_like) FROM weather
2025-03-02 09:07:20,744 INFO sqlalchemy.engine.Engine [generated in 0.00136s] {}


[(7.619449999999999,)]

In [22]:
# Average weather
stmt = text("SELECT city FROM avg_forecast WHERE feels_like > 10")
conn = engine.connect()
result = conn.execute(stmt)
result.fetchall()

2025-03-02 09:07:20,931 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-02 09:07:20,933 INFO sqlalchemy.engine.Engine SELECT city FROM avg_forecast WHERE feels_like > 10
2025-03-02 09:07:20,936 INFO sqlalchemy.engine.Engine [generated in 0.00469s] {}


[('Aix en Provence',),
 ('Bayonne',),
 ('Carcassonne',),
 ('Collioure',),
 ('La Rochelle',),
 ('Marseille',)]

In [23]:
# Top 5 cities
stmt = text("SELECT city,feels_like FROM top_5_cities")
conn = engine.connect()
result = conn.execute(stmt)
result.fetchall()

2025-03-02 09:07:21,133 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-02 09:07:21,134 INFO sqlalchemy.engine.Engine SELECT city,feels_like FROM top_5_cities
2025-03-02 09:07:21,135 INFO sqlalchemy.engine.Engine [generated in 0.00153s] {}


[('Aix en Provence', 11.8065),
 ('Carcassonne', 10.91525),
 ('Bayonne', 10.75925),
 ('Marseille', 10.62475),
 ('La Rochelle', 10.61)]